### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def free_storage(x):
    x.storage().resize_(0)

In [ ]:
class Copy(torch.autograd.Function):
    @staticmethod
    def forward(ctx, prev_stream, next_stream, x):
        ctx.prev_stream = prev_stream
        ctx.next_stream = next_stream
        
        compute_stream = torch.cuda.default_stream(next_stream.device)
        
        with torch.cuda.stream(prev_stream), torch.cuda.stream(next_stream):
            moved_x = x.to(next_stream.device)
            
            x.record_stream(prev_stream)
            moved_x.record_stream(compute_stream)
        
        return moved_x
    
    @staticmethod
    def backward(ctx, grad_input):
        prev_stream = ctx.prev_stream
        next_stream = ctx.next_stream
        
        compute_stream = torch.cuda.default_stream(prev_stream.device)
        
        with torch.cuda.stream(prev_stream), torch.cuda.stream(next_stream):
            moved_grad = grad_input.to(prev_stream.device)
            
            grad_input.record_stream(next_stream)
            moved_grad.record_stream(compute_stream)
        
        return (None, None, moved_grad)

In [ ]:
ready, running, suceed, failed, cooldown, blacklisted

In [2]:
import torch.distributed as dist
import torch.distributed.rpc as rpc

In [ ]:
class DataParallel:
    def __init__(self, module, parallel_context):
        self.module = module
        self.parallel_context = parallel_context
    
    def parallelize(self):
        module = self.module
        
        if self.parallel_context.data_parallel_size > 1:
            for p in module.parameters():
                if p.requires_grad:
                    p.register_hook(_average_grad_hook)
        
        return module
    
    def _average_grad_hook(p):
        data_parallel_size = self.parallel_context.data_parallel_size
        process_group = self.parallel_context.get_group(ParallelMode.DATA)
        p /= data_parallel_size
        
        dist.all_reduce(p, op=dist.ReduceOp.SUM, group=process_group)
        
        return p

In [ ]:
step 1: make 

In [ ]:
min(clock_idx+1, n_partitions)

In [ ]:
process_group, ranks in group, local_rank, 

In [ ]:
step 1:
step 2: init global distributed group
step 3: init parallel groups
step 4: set device
step 5: set seed 

In [ ]:
broadcast > gather > scatter > all-reduce

In [ ]:
not_empty = (linear_probe[..., 1] + linear_probe[..., 2]) / 2

In [ ]:
empty_vs_not_empty = linear_probe[.., 0] - not_empty

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
embed = cache[hook_name]

In [ ]:
embed[:, -1] @ 

In [ ]:
target_tokens = tokens[1:]
W_U_targets = model.W_U[:, target_tokens]

In [3]:
from einops import einsum

In [ ]:
einsum(
    embed, W_U_targets
)

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
ln0 = model.blocks[0].ln1
mlp = model.blocks[0].mlp
embed = model.embed

In [ ]:
embeddings = embed(tokens)

In [ ]:
emebddings = embeddings + mlp(ln0(tokens))

tensor_parallel_size, pipeline_parallel_size, data_parallel_size, host, port

In [ ]:
step 1: mask input tokens
step 2: local embeddings
step 3: global embeddings


In [ ]:
job selector > spawn initial workers > pool watcher

In [ ]:
step 1: autograd
step 2: user
step 3: pytorch
step 4: trigger
step 5: run

In [5]:
from typing import cast, List

In [ ]:
numbers = cast(List[int], numbers)